In [1]:
import pandas as pd
from sqlalchemy import create_engine
import json
import psycopg2

In [2]:
connection_string = psycopg2.connect(host="mypostgresdb.cnm9emj4w74o.us-east-2.rds.amazonaws.com", port = 5432, database="project3", user="rootlh", password="password")

In [ ]:
# cur = connection_string.cursor()

In [ ]:
# cur.execute("""SELECT * FROM posts_info""")
# query_results = cur.fetchall()
# query_results

In [3]:
df = pd.read_sql_query('select * from post_info', con=connection_string)
df.head()

,type,posts
0,INTJ,'https://www.youtube.com/watch?v=8IEQpfA528M ...
1,INFP,"'It's one of my daily 'music doodles', one min..."
2,INFP,'NO. But I have dyscalculia. :X|||I've learned...
3,ISTP,"'288970 It's been a while, guys. Also I bleach..."
4,INFP,'29 and still have it. Never had a girlfriend....


In [4]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)

df['upper_case'] = df['posts'].str.findall(r'[A-Z]').str.len()

df['question'] = df['posts'].apply(lambda x: x.count('?')/50)
df['exclamation'] = df['posts'].apply(lambda x: x.count('!')/50)
df['ellipsis'] = df['posts'].apply(lambda x: x.count('...')/50)

df['emojis'] = df['posts'].str.findall(u'[\U0001f300-\U0001f650]|[\u2000-\u3000]').str.len()

print(df.head())

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86   
1  INFP  'It's one of my daily 'music doodles', one min...              17.46   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30   
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90   
4  INFP  '29 and still have it. Never had a girlfriend....              27.98   

   upper_case  question  exclamation  ellipsis  emojis  
0         182      0.68         0.00      0.60       3  
1         234      0.30         0.12      0.34       0  
2         184      0.12         0.06      0.32       0  
3         125      0.16         0.26      0.10       0  
4         221      0.16         0.06      0.80       0  


In [5]:
map1 = {"I": 0, "E": 1}
map2 = {"N": 0, "S": 1}
map3 = {"T": 0, "F": 1}
map4 = {"J": 0, "P": 1}
df['I-E'] = df['type'].astype(str).str[0]
df['I-E'] = df['I-E'].map(map1)
df['N-S'] = df['type'].astype(str).str[1]
df['N-S'] = df['N-S'].map(map2)
df['T-F'] = df['type'].astype(str).str[2]
df['T-F'] = df['T-F'].map(map3)
df['J-P'] = df['type'].astype(str).str[3]
df['J-P'] = df['J-P'].map(map4)
print(df.head(10))

   type                                              posts  words_per_comment  \
0  INTJ  'https://www.youtube.com/watch?v=8IEQpfA528M  ...              22.86   
1  INFP  'It's one of my daily 'music doodles', one min...              17.46   
2  INFP  'NO. But I have dyscalculia. :X|||I've learned...              13.30   
3  ISTP  '288970 It's been a while, guys. Also I bleach...              10.90   
4  INFP  '29 and still have it. Never had a girlfriend....              27.98   
5  INFJ  'It gets to be pretty obvious the lifestyle di...              20.98   
6  INTP  'I used to find this difficult until I realize...              23.80   
7  INTJ  '1. ENTJs - Without fail my favorite people.  ...              33.88   
8  ENFP  'Can I just say I'm so glad and thankful that ...              26.28   
9  ESFP  'No I'm not, you go home!|||Farnsworth: ENTP F...              21.80   

   upper_case  question  exclamation  ellipsis  emojis  I-E  N-S  T-F  J-P  
0         182      0.68        

In [6]:
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import os
import seaborn as sns
from matplotlib import style
style.use("ggplot")

In [ ]:
# plt.figure(figsize=(15,10))
# sns.jointplot(x='words_per_comment', y='upper_case', data=df, kind='kde')

In [ ]:
# plt.figure(figsize=(15,10))
# sns.jointplot(x='words_per_comment', y='emojis', data=df, kind='kde')

In [7]:
X = df.drop(['type','posts','I-E','N-S','T-F','J-P'], axis=1).values
y = df['type'].values

print(y.shape)
print(X.shape)

(8675,)
(8675, 6)


In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.1, random_state=5)

In [34]:
from sklearn.svm import SVC 
model = SVC(kernel='sigmoid',gamma='auto')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='sigmoid',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [35]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.219


In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=42)

In [45]:
from sklearn.svm import SVC 
model = SVC(kernel='rbf',gamma='scale')
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [46]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.225


In [47]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        ENFJ       0.00      0.00      0.00        18
        ENFP       0.00      0.00      0.00        52
        ENTJ       0.00      0.00      0.00        23
        ENTP       0.00      0.00      0.00        64
        ESFJ       0.00      0.00      0.00         6
        ESFP       0.00      0.00      0.00         3
        ESTJ       0.00      0.00      0.00         3
        ESTP       0.00      0.00      0.00        14
        INFJ       0.20      0.03      0.04       158
        INFP       0.23      1.00      0.37       191
        INTJ       0.00      0.00      0.00       106
        INTP       0.00      0.00      0.00       131
        ISFJ       0.00      0.00      0.00        19
        ISFP       0.00      0.00      0.00        21
        ISTJ       0.00      0.00      0.00        21
        ISTP       0.00      0.00      0.00        38

    accuracy                           0.22       868
   macro avg       0.03   

C:\Users\hph77\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
